# MLOps Walkthrough - Model Evaluation

key principles / tasks supported by RSEs
* reusable components - making model results available to other researchers
* running at scale - serving many results
* interactive exploration of results (predictions and XAI)



## Example Load in trained model and do inference

use mlflow for iinference

# Example - calculate feature importance and vosualise results

### Example - Running a cluster with ray serve to serve many results

### Example - create a dashboard with holoviz that gets inference results from a ray serve or mlflow serve?

### Reference

* [Holoviz](https://holoviz.org/) 
* SHAP
* 